In [ ]:
import os
import json
import numpy as np
import pandas as pd
from tqdm import tqdm

# ================= 경로 =================
KP_ROOT = "/content/drive/MyDrive/[Projects]/AI Hub/Pose Detection 기반 실시간 낙상 감지 시스템 개발/Data/data_resized/낙상사고 위험동작 영상-센서 쌍 데이터_병원,후면낙상/3.개방데이터/1.데이터/Training/01.원천데이터/TS/kp"
BBOX_ROOT = "/content/drive/MyDrive/[Projects]/AI Hub/Pose Detection 기반 실시간 낙상 감지 시스템 개발/Data/data_resized/낙상사고 위험동작 영상-센서 쌍 데이터_병원,후면낙상/3.개방데이터/1.데이터/Training/02.라벨링데이터/TL/이미지/bbox_motion"
VIDEO_FEAT_PATH = "/content/drive/MyDrive/[Projects]/AI Hub/Pose Detection 기반 실시간 낙상 감지 시스템 개발/Data/data/낙상사고 위험동작 영상-센서 쌍 데이터_병원,후면낙상/3.개방데이터/1.데이터/Training/01.원천데이터/TS/영상/video_motion_features.json"

OUT_CSV = "/content/drive/MyDrive/[Projects]/AI Hub/Pose Detection 기반 실시간 낙상 감지 시스템 개발/Data/data_resized/낙상사고 위험동작 영상-센서 쌍 데이터_병원,후면낙상/3.개방데이터/1.데이터/Training/fall_window_dataset.csv"
T = 10

# ================= load meta =================
with open(VIDEO_FEAT_PATH) as f:
    video_feats = json.load(f)

bbox_scene_set = set(f.replace(".json", "") for f in os.listdir(BBOX_ROOT))
video_scene_set = set(video_feats.keys())

# ================= skeleton loader =================
def load_kp(path):
    with open(path) as f:
        data = json.load(f)
    kp = data.get("keypoints", [])
    if not kp or not kp[0] or not kp[0][0]:
        return None
    arr = np.array(kp[0][0], dtype=np.float32)
    if arr.shape != (17, 3):
        return None
    return arr

# ================= column names =================
columns = ["scene_id", "label"]

for t in range(T):
    for i in range(17):
        for k in ["x", "y", "c"]:
            columns.append(f"sk_{i}_{k}_f{t}")

for t in range(T):
    for k in ["cx", "cy", "w", "h", "dx", "dy"]:
        columns.append(f"bbox_{k}_f{t}")

columns += ["vid_mean_flow", "vid_max_flow", "vid_mean_diff"]

rows = []

LABEL_MAP = {
    "N": ("N", 0),
    "Y": ("SY", 1)
}

# ================= main =================
for label_name, (subdir, label) in LABEL_MAP.items():
    label_root = os.path.join(KP_ROOT, label_name, subdir)
    if not os.path.exists(label_root):
        continue

    for scene in tqdm(os.listdir(label_root), desc=f"CSV {label_name}"):
        if scene not in bbox_scene_set or scene not in video_scene_set:
            continue

        scene_dir = os.path.join(label_root, scene)
        kp_files = sorted(f for f in os.listdir(scene_dir) if f.endswith(".json"))
        if len(kp_files) != T:
            continue

        row = [scene, label]

        # ===== skeleton =====
        valid = True
        for kf in kp_files:
            kp = load_kp(os.path.join(scene_dir, kf))
            if kp is None:
                valid = False
                break
            row.extend(kp.flatten().tolist())

        if not valid:
            continue

        # ===== bbox =====
        with open(os.path.join(BBOX_ROOT, f"{scene}.json")) as f:
            bbox = json.load(f)["bbox_motion"]
        if len(bbox) != T:
            continue

        for b in bbox:
            row.extend(b)

        # ===== video =====
        vf = video_feats[scene]
        row.extend([vf["mean_flow"], vf["max_flow"], vf["mean_diff"]])

        rows.append(row)

# ================= save =================
df = pd.DataFrame(rows, columns=columns)
df.to_csv(OUT_CSV, index=False)

print("✅ CSV dataset created")
print(df.shape)

CSV Y: 100%|██████████| 1856/1856 [4:00:56<00:00,  7.79s/it]


✅ CSV dataset created
(3951, 575)


In [ ]:
import os
import json
import numpy as np
import pandas as pd
from tqdm import tqdm

# ================= 경로 =================
KP_ROOT = "/content/drive/MyDrive/[Projects]/AI Hub/Pose Detection 기반 실시간 낙상 감지 시스템 개발/Data/data_resized/낙상사고 위험동작 영상-센서 쌍 데이터_병원,후면낙상/3.개방데이터/1.데이터/Validation/01.원천데이터/VS/kp"
BBOX_ROOT = "/content/drive/MyDrive/[Projects]/AI Hub/Pose Detection 기반 실시간 낙상 감지 시스템 개발/Data/data_resized/낙상사고 위험동작 영상-센서 쌍 데이터_병원,후면낙상/3.개방데이터/1.데이터/Validation/02.라벨링데이터/VL/이미지/bbox_motion"
VIDEO_FEAT_PATH = "/content/drive/MyDrive/[Projects]/AI Hub/Pose Detection 기반 실시간 낙상 감지 시스템 개발/Data/data/낙상사고 위험동작 영상-센서 쌍 데이터_병원,후면낙상/3.개방데이터/1.데이터/Validation/01.원천데이터/VS/영상/video_motion_features.json"

OUT_CSV = "/content/drive/MyDrive/[Projects]/AI Hub/Pose Detection 기반 실시간 낙상 감지 시스템 개발/Data/data_resized/낙상사고 위험동작 영상-센서 쌍 데이터_병원,후면낙상/3.개방데이터/1.데이터/Validation/fall_window_dataset.csv"
T = 10

# ================= load meta =================
with open(VIDEO_FEAT_PATH) as f:
    video_feats = json.load(f)

bbox_scene_set = set(f.replace(".json", "") for f in os.listdir(BBOX_ROOT))
video_scene_set = set(video_feats.keys())

# ================= skeleton loader =================
def load_kp(path):
    with open(path) as f:
        data = json.load(f)
    kp = data.get("keypoints", [])
    if not kp or not kp[0] or not kp[0][0]:
        return None
    arr = np.array(kp[0][0], dtype=np.float32)
    if arr.shape != (17, 3):
        return None
    return arr

# ================= column names =================
columns = ["scene_id", "label"]

for t in range(T):
    for i in range(17):
        for k in ["x", "y", "c"]:
            columns.append(f"sk_{i}_{k}_f{t}")

for t in range(T):
    for k in ["cx", "cy", "w", "h", "dx", "dy"]:
        columns.append(f"bbox_{k}_f{t}")

columns += ["vid_mean_flow", "vid_max_flow", "vid_mean_diff"]

rows = []

LABEL_MAP = {
    "N": ("N", 0),
    "Y": ("SY", 1)
}

# ================= main =================
for label_name, (subdir, label) in LABEL_MAP.items():
    label_root = os.path.join(KP_ROOT, label_name, subdir)
    if not os.path.exists(label_root):
        continue

    for scene in tqdm(os.listdir(label_root), desc=f"CSV {label_name}"):
        if scene not in bbox_scene_set or scene not in video_scene_set:
            continue

        scene_dir = os.path.join(label_root, scene)
        kp_files = sorted(f for f in os.listdir(scene_dir) if f.endswith(".json"))
        if len(kp_files) != T:
            continue

        row = [scene, label]

        # ===== skeleton =====
        valid = True
        for kf in kp_files:
            kp = load_kp(os.path.join(scene_dir, kf))
            if kp is None:
                valid = False
                break
            row.extend(kp.flatten().tolist())

        if not valid:
            continue

        # ===== bbox =====
        with open(os.path.join(BBOX_ROOT, f"{scene}.json")) as f:
            bbox = json.load(f)["bbox_motion"]
        if len(bbox) != T:
            continue

        for b in bbox:
            row.extend(b)

        # ===== video =====
        vf = video_feats[scene]
        row.extend([vf["mean_flow"], vf["max_flow"], vf["mean_diff"]])

        rows.append(row)

# ================= save =================
df = pd.DataFrame(rows, columns=columns)
df.to_csv(OUT_CSV, index=False)

print("✅ CSV dataset created")
print(df.shape)

CSV Y: 100%|██████████| 280/280 [35:45<00:00,  7.66s/it]


✅ CSV dataset created
(563, 575)
